In [19]:
import sys
import warnings
warnings.filterwarnings("ignore")

import os
import codecs
import json
import cv2
from keras.models import load_model
import numpy as np
from keras.preprocessing import image
from tqdm import tqdm 
from utils.datasets import get_labels
from utils.inference import detect_faces
from utils.inference import draw_text
from utils.inference import draw_bounding_box
from utils.inference import apply_offsets
from utils.inference import load_detection_model
from utils.inference import load_image
from utils.preprocessor import preprocess_input
from keras.models import Model
import tensorflow as tf

In [24]:
data_path='../../Dataset/img_align_celeba/'

In [2]:
# hyper-parameters for bounding boxes shape
gender_offsets = (30, 60)
gender_offsets = (10, 10)
emotion_offsets = (20, 40)
emotion_offsets = (0, 0)

### Loading labels and Pre-trained models

In [3]:
detection_model_path = '../trained_models/detection_models/haarcascade_frontalface_default.xml'
emotion_model_path = '../trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5'
gender_model_path = '../trained_models/gender_models/simple_CNN.81-0.96.hdf5'

In [4]:
emotion_labels = get_labels('fer2013')
gender_labels = get_labels('imdb')

In [5]:
face_detection = load_detection_model(detection_model_path)
emotion_classifier = load_model(emotion_model_path, compile=False)
gender_classifier = load_model(gender_model_path, compile=False)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# getting input model shapes for inference
emotion_target_size = emotion_classifier.input_shape[1:3]
gender_target_size = gender_classifier.input_shape[1:3]

In [7]:
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
def converter(x):

    #x has shape (batch, width, height, channels)
    return (0.21 * x[:,:,:1]) + (0.72 * x[:,:,1:2]) + (0.07 * x[:,:,-1:])

### Customizing Prediciton Functions (To ensure compatibility with LIME)

In [8]:
target_size=None
from skimage import io
from skimage.color import rgb2gray
from skimage import img_as_ubyte
import random

In [9]:
def gender_predict(img):
    gray_image=converter(img)
    rgb_image=np.squeeze(img)
    rgb_image = rgb_image.astype('uint8')
    
    gray_image = np.squeeze(gray_image)
    gray_image = gray_image.astype('uint8')
    faces = detect_faces(face_detection, gray_image)

    if len(faces)==0:
        rand_index=random.randint(0,1)
        ret_arr=np.zeros((1,2))
        ret_arr[0][rand_index]=1.0
        return ret_arr
    x1, x2, y1, y2 = apply_offsets(faces[0], gender_offsets)
    
    rgb_face = rgb_image[y1:y2, x1:x2]
    rgb_face = cv2.resize(rgb_face, (gender_target_size))

    rgb_face = preprocess_input(rgb_face, False)
    rgb_face = np.expand_dims(rgb_face, 0)
    
    return gender_classifier.predict(rgb_face)

In [10]:
def emotion_predict(img):
    gray_image=converter(img)
    # print(gray_image.shape)
    gray_image = np.squeeze(gray_image)
    gray_image = gray_image.astype('uint8')
    faces = detect_faces(face_detection, gray_image)

    if len(faces)==0:
        rand_index=random.randint(0,6)
        ret_arr=np.zeros((1,7))
        ret_arr[0][rand_index]=1.0
        return ret_arr
    
    x1, x2, y1, y2 = apply_offsets(faces[0], emotion_offsets)
    gray_face = gray_image[y1:y2, x1:x2]
    gray_face = cv2.resize(gray_face, (emotion_target_size))

    gray_face = preprocess_input(gray_face, True)
    gray_face = np.expand_dims(gray_face, 0)
    gray_face = np.expand_dims(gray_face, -1)
    return emotion_classifier.predict(gray_face)

### Storing Gender Predictions

In [55]:
output_dict={}
for file in tqdm(os.listdir(data_path)):
    rgb_img = io.imread(os.path.join(data_path,file))
    output=gender_predict(rgb_img)[0]
    output_dict[file]={gender_labels[i]:np.round(output[i]*100,2) for i in range(len(gender_labels))}

100%|██████████| 1000/1000 [00:33<00:00, 30.02it/s]


In [56]:
import json

with open('gender_results_1000.json', 'w') as fp:
    json.dump(output_dict, fp)

### Storing Emotion Predictions

In [57]:
output_dict={}
for file in tqdm(os.listdir(data_path)):
    rgb_img = io.imread(os.path.join(data_path,file))
    output=emotion_predict(rgb_img)[0]
    output_dict[file]={emotion_labels[i]:np.round(output[i]*100,2) for i in range(len(emotion_labels))}

100%|██████████| 1000/1000 [00:31<00:00, 31.67it/s]


In [61]:
import json

with open('emotion_results_1000.json', 'w') as fp:
    json.dump(output_dict, fp)

### LIME Masks

In [11]:
import lime 
from lime import lime_image
from math import ceil,floor
from skimage.measure import block_reduce
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
%matplotlib inline
explainer = lime_image.LimeImageExplainer()

In [12]:
def generate_mask_gender(image):
    np.random.seed(16)
    explanation = explainer.explain_instance(image, gender_predict, top_labels=2, hide_color=0, num_samples=100,batch_size=1)
    ret_list=[]
    for label in explanation.top_labels:
        temp, mask = explanation.get_image_and_mask(label, positive_only=True, num_features=6, hide_rest=False)
        reduced_mask=block_reduce(mask, block_size=(7, 7), func=np.max)
        ret_list+=[(reduced_mask,reduced_mask.tolist(),label)]
    return ret_list

def generate_mask_emotion(image):
    np.random.seed(16)
    explanation = explainer.explain_instance(image, emotion_predict, top_labels=7, hide_color=0, num_samples=100,batch_size=1)
    ret_list=[]
    for label in explanation.top_labels:
        temp, mask = explanation.get_image_and_mask(label, positive_only=True, num_features=6, hide_rest=False)
        reduced_mask=block_reduce(mask, block_size=(7, 7), func=np.max)
        ret_list+=[(reduced_mask,reduced_mask.tolist(),label)]
    return ret_list

In [13]:
for image in tqdm(os.listdir(data_path)):
    rgb_img=io.imread(os.path.join(data_path,image))
    emotion_masks=generate_mask_emotion(rgb_img)
    gender_masks=generate_mask_gender(rgb_img)
    outfile='../../lime_masks/'+image.split('.')[0]+'.json'
    image_output={}
    for arr,arr_list,label in emotion_masks:
        image_output[emotion_labels[label]]=arr_list
    
    for arr,arr_list,label in gender_masks:
        image_output[gender_labels[label]]=arr_list
        
    json.dump(image_output, codecs.open(outfile, 'w', encoding='utf-8'), sort_keys=True, indent=4)

100%|██████████| 1000/1000 [20:38<00:00,  1.03it/s]


### Generate Emotion Predictions in D3 plot format

In [14]:
format_dict=json.load(open('example_format.json','r'))

In [15]:
format_dict['children']

In [16]:
emotion_dict=json.load(open('emotion_results_1000.json','r'))

In [17]:
for image in tqdm(emotion_dict):
    outfile='../../emotion_predictions/'+image.split('.')[0]+'.json'
    for dictionary in format_dict['children']:
        emotion_label=dictionary['name']
        dictionary['value']=np.round(emotion_dict[image][emotion_label]/100,2)
    json.dump(format_dict, codecs.open(outfile, 'w', encoding='utf-8'), indent=4)

### Feature Extractor for TSNE visuals

In [29]:
from sklearn.decomposition import PCA
import pickle

In [31]:
genfeat_extractor = Model(inputs=gender_classifier.input, outputs=gender_classifier.get_layer("conv2d_8").output)
emofeat_extractor = Model(inputs=emotion_classifier.input, outputs=emotion_classifier.get_layer("conv2d_7").output)
images=[os.path.join(data_path,img) for img in os.listdir(data_path)]

In [32]:
def gender_feature_extractor(img):
    gray_image=converter(img)
    rgb_image=np.squeeze(img)
    rgb_image = rgb_image.astype('uint8')
    
    gray_image = np.squeeze(gray_image)
    gray_image = gray_image.astype('uint8')
    faces = detect_faces(face_detection, gray_image)

    if len(faces)==0:
        sampl=np.random.uniform(low=-35.5, high=35.5, size=(1,2304))
        return sampl
    
    x1, x2, y1, y2 = apply_offsets(faces[0], gender_offsets)
    
    rgb_face = rgb_image[y1:y2, x1:x2]
    rgb_face = cv2.resize(rgb_face, (gender_target_size))

    rgb_face = preprocess_input(rgb_face, False)
    rgb_face = np.expand_dims(rgb_face, 0)
    
    feat_out=genfeat_extractor.predict(rgb_face)[0]
    feat_out=np.array([np.ndarray.flatten(feat_out)])
    return feat_out

In [33]:
def emotion_feature_extractor(img):
    gray_image=converter(img)
    # print(gray_image.shape)
    gray_image = np.squeeze(gray_image)
    gray_image = gray_image.astype('uint8')
    faces = detect_faces(face_detection, gray_image)

    if len(faces)==0:
        sampl=np.random.uniform(low=-35.5, high=35.5, size=(1,112))
        return sampl
    
    x1, x2, y1, y2 = apply_offsets(faces[0], emotion_offsets)
    gray_face = gray_image[y1:y2, x1:x2]
    gray_face = cv2.resize(gray_face, (emotion_target_size))

    gray_face = preprocess_input(gray_face, True)
    gray_face = np.expand_dims(gray_face, 0)
    gray_face = np.expand_dims(gray_face, -1)
    feat_out=emofeat_extractor.predict(gray_face)[0]
    # feat_out.shape
    feat_out=np.array([np.ndarray.flatten(feat_out)])
    return feat_out

In [34]:
def generate_feature_list(model='gender'):
    feature_list=[]
    image_list=[]
    error=[]
    for elem in tqdm(os.listdir(data_path)):
        rgb_img = io.imread(os.path.join(data_path,elem))
        if model=='emotion':
            output=emotion_feature_extractor(rgb_img)
        else:
            output=gender_feature_extractor(rgb_img)
        image_list.append(elem)
        feature_list.append(list(output[0]))
    return feature_list

In [36]:
model='gender'
n_comp=100
features = np.array(generate_feature_list(model))
pca = PCA(n_components=n_comp)
pca.fit(features)
pca_features = pca.transform(features)
pickle.dump([images, pca_features, pca], open('../../tSNE/assets/work/pcafeatures_gender.p', 'wb'))

100%|██████████| 1000/1000 [00:10<00:00, 92.69it/s]


In [37]:
model='emotion'
n_comp=50
features = np.array(generate_feature_list(model))
pca = PCA(n_components=n_comp)
pca.fit(features)
pca_features = pca.transform(features)
pickle.dump([images, pca_features, pca], open('../../tSNE/assets/work/pcafeatures_emotion.p', 'wb'))

100%|██████████| 1000/1000 [00:13<00:00, 72.67it/s]


### Generating TSNE Images

In [28]:
### Run the image-tsne.ipynb file using the appropriate pickle file generated above